## Load Packages

In [1]:
using Distributions, Roots, QuadGK
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Black-Scholes

The Black-Scholes formula for a European call option on an asset with a continuous dividend rate $\delta$

$C  =e^{-\delta m}S\Phi\left(  d_{1}\right)  -e^{-ym}K\Phi\left(d_{2}\right)$, where

$d_{1} =\frac{\ln(S/K)+(y-\delta+\sigma^{2}/2)m}{\sigma\sqrt{m}} \ \text{ and } \ d_{2}=d_{1}-\sigma\sqrt{m}$ 

and where $\Phi(d)$ denotes the probability of $x\leq d$ when $x$ has an $N(0,1)$ distribution.

In [3]:
function Φ(x)
    #Calculates Pr(z<=x) for N(0,1) variable z
    Pr = cdf(Normal(0,1),x)
  return Pr
end

function OptionBlackSPs(S,K,m,y,σ,δ=0,PutIt=false)
    #Calculates Black-Scholes European call or put option price, continuous dividends of δ
    d1 = ( log.(S./K) + (y-δ+0.5*σ^2)*m ) / (σ*sqrt(m))
    d2 = d1 - σ*sqrt(m)
    c  = exp(-δ*m)*S.*Φ.(d1) - K.*exp(-y*m).*Φ.(d2)
    if PutIt 
        price = c - exp(-δ*m)*S + exp(-y*m)*K    
    else     
        price = c
    end    
    return price      
end

OptionBlackSPs (generic function with 3 methods)

In [4]:
S = 42
K = 42
m = 0.5
y = 0.05
σ = 0.2

C = OptionBlackSPs(S,K,m,y,σ)
printlnPs("\n","call price according to Black-Scholes at K=$K: ",C)

P = OptionBlackSPs(S,K,m,y,σ,0,true)
printlnPs("\n","put price according to Black-Scholes at K=$K: ",P)

δ = 0.05
Pδ = OptionBlackSPs(S,K,m,y,σ,δ,true)
printlnPs("\n","put price according to Black-Scholes at K=$K and δ=$δ: ",Pδ)


call price according to Black-Scholes at K=42:      2.893

put price according to Black-Scholes at K=42:      1.856

put price according to Black-Scholes at K=42 and δ=0.05:      2.309


In [5]:
KM = 30:60
C_K = OptionBlackSPs(S,KM,m,y,σ)        #at different strike prices 

SM = 30:60
C_S = OptionBlackSPs(SM,K,m,y,σ)        #at different spot prices

plot(KM,C_K,color=:red,legend=false)
title!("Black-Scholes call option price (different K)")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 35 
 
 
 40 
 
 
 45 
 
 
 50 
 
 
 55 
 
 
 60 
 
 
 5 
 
 
 10 
 
 
 Black-Scholes call option price (different K) 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 57.7953,35.2609 76.2205,62.4465 94.6457,89.3181 113.071,115.729 131.496,141.5 149.921,166.429 168.346,190.299 186.772,212.895 205.197,234.021 
 223.622,253.511 242.047,271.245 260.472,287.154 278.898,301.22 297.323,313.481 315.748,324.017 334.173,332.943 352.598,340.405 371.024,346.558 389.449,351.568 
 407.874,355.597 426.299,358.798 444.724,361.313 463.15,363.267 481.575,364.77 500,365.913 518.425,366.776 536.85,367.42 555.276,367.897 573.701,368.248 
 592.126,368.504 
 "/>

In [6]:
plot(SM,C_S,color=:red,legend=false)
title!("Black-Scholes call option price (different S)")
xlabel!("asset price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 35 
 
 
 40 
 
 
 45 
 
 
 50 
 
 
 55 
 
 
 60 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Black-Scholes call option price (different S) 
 
 
 asset price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 57.7953,368.083 76.2205,367.377 94.6457,366.257 113.071,364.577 131.496,362.173 149.921,358.882 168.346,354.551 186.772,349.052 205.197,342.29 
 223.622,334.211 242.047,324.806 260.472,314.102 278.898,302.166 297.323,289.09 315.748,274.985 334.173,259.973 352.598,244.179 371.024,227.724 389.449,210.72 
 407.874,193.268 426.299,175.458 444.724,157.365 463.15,139.051 481.575,120.568 500,101.956 518.425,83.2487 536.85,64.47 555.276,45.639 573.701,26.7701 
 592.126,7.87402 
 "/>

# Implied Volatility

Idea: use an observed option price and solve the BS formula for the unknown $\sigma$

In [7]:
σM = 0.01:0.01:0.3

C_σ = fill(NaN,length(σM))
for i = 1:length(σM)
    C_σ[i] = OptionBlackSPs(S,K,m,y,σM[i])        #at different strike prices 
end


plot(σM,C_σ,color=:red,label="BS price")
title!("Black-Scholes call option price (different sigmas)")
xlabel!("volatility (sigma)")
ylabel!("option price")
plot!([C;C+0.5;C-0.5],linetype=:hline,linecolor=:black,line=(:dash,1),label="data on prices ")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.1 
 
 
 0.2 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 Black-Scholes call option price (different sigmas) 
 
 
 volatility (sigma) 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 58.4306,367.418 77.4912,362.325 96.5517,353.967 115.612,343.702 134.673,332.318 153.733,320.238 172.794,307.702 191.854,294.853 210.915,281.78 
 229.976,268.543 249.036,255.182 268.097,241.726 287.157,228.194 306.218,214.603 325.278,200.963 344.339,187.284 363.399,173.574 382.46,159.836 401.52,146.078 
 420.581,132.3 439.642,118.508 458.702,104.704 477.763,90.8889 496.823,77.0657 515.884,63.2358 534.944,49.4005 554.005,35.5611 573.065,21.7186 592.126,7.87402 
 
 "/>
 
 
 
 
 
 
 
 BS price 
 
 
 
 data on prices

In [8]:
println("Compare these results with the previous graph\n")

iv = fzero(sigma->OptionBlackSPs(S,K,m,y,sigma)-C,[-1;1])
printlnPs("Implied volatility (should be the same as above): ",iv)

iv_a = fzero(sigma->OptionBlackSPs(S,K,m,y,sigma)-(C+0.5),[-1;1])
printlnPs("Implied volatility (overpriced option): ",iv_a)

iv_b = fzero(sigma->OptionBlackSPs(S,K,m,y,sigma)-(C-0.5),[-1;1])
printlnPs("Implied volatility (underpriced option): ",iv_b)

Compare these results with the previous graph

Implied volatility (should be the same as above):      0.200
Implied volatility (overpriced option):      0.243
Implied volatility (underpriced option):      0.156


# BS from an Explicit Integration

The price of a European a call option is 
$$
C=e^{-ym}\operatorname*{E}\nolimits^{\ast}\max(0,S_{m}-K),
$$
which can be written

$$
C=e^{-ym}\int\nolimits_{K}^{\infty}\left(  S_{m}-K\right)  f^{\ast}(S_{m})dS_{m},
$$

where $f^{\ast}(S_{m})$ is the risk neutral density function of the asset
price at expiration ($S_{m}$). (Below $K$ the value of the integrand is zero.)

For the Black-Scholes model, the (physical) normal distribution for the log
asset price implies that the risk neutral distribution of $\ln S_{m}$ is
$$
\ln S_{m}\sim^{\ast}\text{N}(\ln S+ym-\sigma^{2}m/2,\sigma^{2}m),
$$
where $S$ is the current asset price.

In [9]:
function ϕNS(x,μ=0,σ²=1)                     #pdf of N(μ,σ²), NS for "non-standard"
    pdfx = pdf(Normal(μ,sqrt(σ²)),x)
    return pdfx
end

function BSintegrand(Sm,S,K,y,m,σ²)        #integratre to get Black-Scholes call price
    μ = log(S) + y*m - m*σ²/2
    f = ϕNS(log.(Sm),μ,m*σ²)./Sm      #log-normal pdf
    y = exp(-y*m)*max.(0,Sm-K).*f
    return y
end

BSintegrand (generic function with 1 method)

In [11]:
x =-3:0.01:3
xb = x[x .<= 1.64]

plot(x,ϕNS(x),color=:red,linewidth=2,legend=false)
plot!(xb,ϕNS(xb),color=:red,linewidth=2,legend=nothing,fill=(0,:blue))
title!("pdf of N(0,1)")
xlabel!("x")
plot!([1.64],linetype=:vline,linecolor=:black,line=(:dash,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 pdf of N(0,1) 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,364.498 40.2913,364.376 41.2126,364.251 42.1339,364.122 43.0551,363.991 43.9764,363.855 44.8976,363.716 45.8189,363.574 46.7402,363.427 47.6614,363.277 
 48.5827,363.123 49.5039,362.965 50.4252,362.803 51.3465,362.637 52.2677,362.466 53.189,362.291 54.1102,362.112 55.0315,361.928 55.9528,361.74 56.874,361.547 
 57.7953,361.349 58.7165,361.146 59.6378,360.938 60.5591,360.725 61.4803,360.507 62.4016,360.284 63.3228,360.055 64.2441,359.821 65.1654,359.581 66.0866,359.335 
 67.0079,359.084 67.9291,358.826 68.8504,358.563 69.7717,358.294 70.6929,358.018 71.6142,357.736 72.5354,357.447 73.4567,357.152 74.378,356.85 75.2992,356.541 
 76.2205,356.225 77.1417,355.903 78.063,355.573 78.9843,355.235 79.9055,354.891 80.8268,354.538 81.748,354.178 82.6693,353.81 83.5906,353.435 84.5118,353.051 
 85.4331,352.659 86.3543,352.259 87.2756,351.85 88.1969,351.433 89.1181,351.007 90.0394,350.572 90.9606,350.128 91.8819,349.675 92.8031,349.213 93.7244,348.741 
 94.6457,348.26 95.5669,347.769 96.4882,347.269 97.4094,346.759 98.3307,346.238 99.252,345.708 100.173,345.167 101.094,344.615 102.016,344.053 102.937,343.481 
 103.858,342.897 104.78,342.303 105.701,341.697 106.622,341.08 107.543,340.452 108.465,339.812 109.386,339.161 110.307,338.498 111.228,337.823 112.15,337.135 
 113.071,336.436 113.992,335.724 114.913,335 115.835,334.264 116.756,333.514 117.677,332.752 118.598,331.977 119.52,331.189 120.441,330.387 121.362,329.572 
 122.283,328.744 123.205,327.903 124.126,327.047 125.047,326.178 125.969,325.295 126.89,324.398 127.811,323.486 128.732,322.561 129.654,321.621 130.575,320.667 
 131.496,319.698 132.417,318.715 133.339,317.716 134.26,316.703 135.181,315.675 136.102,314.632 137.024,313.574 137.945,312.501 138.866,311.413 139.787,310.309 
 140.709,309.19 141.63,308.055 142.551,306.905 143.472,305.739 144.394,304.557 145.315,303.36 146.236,302.147 147.157,300.918 148.079,299.673 149,298.412 
 149.921,297.136 150.843,295.843 151.764,294.534 152.685,293.21 153.606,291.869 154.528,290.512 155.449,289.139 156.37,287.75 157.291,286.345 158.213,284.924 
 159.134,283.487 160.055,282.034 160.976,280.564 161.898,279.079 162.819,277.577 163.74,276.06 164.661,274.527 165.583,272.978 166.504,271.413 167.425,269.832 
 168.346,268.235 169.268,266.623 170.189,264.996 171.11,263.352 172.031,261.694 172.953,260.02 173.874,258.331 174.795,256.627 175.717,254.907 176.638,253.173 
 177.559,251.425 178.48,249.661 179.402,247.883 180.323,246.091 181.244,244.284 182.165,242.464 183.087,240.629 184.008,238.781 184.929,236.919 185.85,235.044 
 186.772,233.156 187.693,231.254 188.614,229.34 189.535,227.413 190.457,225.474 191.378,223.523 192.299,221.56 193.22,219.585 194.142,217.599 195.063,215.601 
 195.984,213.593 196.906,211.574 197.827,209.544 198.748,207.504 199.669,205.455 200.591,203.396 201.512,201.327 202.433,199.25 203.354,197.164 204.276,195.069 
 205.197,192.967 206.118,190.856 207.039,188.739 207.961,186.614 208.882,184.482 209.803,182.345 210.724,180.201 211.646,178.052 212.567,175.897 213.488,173.737 
 214.409,171.573 215.331,169.405 216.252,167.233 217.173,165.058 218.094,162.879 219.016,160.698 219.937,158.516 220.858,156.331 221.78,154.145 222.701,151.958 
 223.622,149.771 224.543,147.584 225.465,145.397 226.386,143.211 227.307,141.026 228.228,138.843 229.15,136.663 230.071,134.485 230.992,132.31 231.913,130.139 
 232.835,127.972 233.756,125.81 234.677,123.652 235.598,121.5 236.52,119.355 237.441,117.215 238.362,115.083 239.283,112.958 240.205,110.841 241.126,108.732 
 242.047,106.633 242.969,104.543 243.89,102.462 244.811,100.393 245.732,98.3337 246.654,96.2862 247.575,94.2506 248.496,92

In [12]:
cdf164, = QuadGK.quadgk(x->ϕNS(x),-Inf,1.64)
printlnPs("\nPr(x<=1.64) according to N(0,1): $(round(cdf164,3))")


Pr(x<=1.64) according to N(0,1): 0.949


In [13]:

plot(30:0.25:100,BSintegrand(30:0.25:100,S,K,y,m,σ^2),color=:red,ylims=(-0.01,0.25),legend=false)
title!("What we integrate to get Black-Scholes call option price")
xlabel!("asset price")
ylabel!("integrand")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 70 
 
 
 80 
 
 
 90 
 
 
 100 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 What we integrate to get Black-Scholes call option price 
 
 
 asset price 
 
 
 integrand 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,354.634 41.3442,354.634 43.3183,354.634 45.2925,354.634 47.2666,354.634 49.2407,354.634 51.2148,354.634 53.189,354.634 55.1631,354.634 57.1372,354.634 
 59.1114,354.634 61.0855,354.634 63.0596,354.634 65.0337,354.634 67.0079,354.634 68.982,354.634 70.9561,354.634 72.9303,354.634 74.9044,354.634 76.8785,354.634 
 78.8526,354.634 80.8268,354.634 82.8009,354.634 84.775,354.634 86.7492,354.634 88.7233,354.634 90.6974,354.634 92.6715,354.634 94.6457,354.634 96.6198,354.634 
 98.5939,354.634 100.568,354.634 102.542,354.634 104.516,354.634 106.49,354.634 108.465,354.634 110.439,354.634 112.413,354.634 114.387,354.634 116.361,354.634 
 118.335,354.634 120.309,354.634 122.283,354.634 124.258,354.634 126.232,354.634 128.206,354.634 130.18,354.634 132.154,354.634 134.128,354.634 136.102,332.099 
 138.076,309.749 140.051,287.696 142.025,266.047 143.999,244.905 145.973,224.363 147.947,204.511 149.921,185.427 151.895,167.185 153.87,149.848 155.844,133.472 
 157.818,118.105 159.792,103.786 161.766,90.5447 163.74,78.405 165.714,67.3818 167.688,57.4825 169.663,48.7076 171.637,41.0508 173.611,34.4994 175.585,29.0349 
 177.559,24.6335 179.533,21.2666 181.507,18.9009 183.481,17.4997 185.456,17.0225 187.43,17.4263 189.404,18.6655 191.378,20.6929 193.352,23.4595 195.326,26.9154 
 197.3,31.0104 199.274,35.6935 201.249,40.9145 203.223,46.623 205.197,52.7698 207.171,59.3064 209.145,66.1859 211.119,73.3626 213.093,80.7923 215.067,88.433 
 217.042,96.2442 219.016,104.188 220.99,112.227 222.964,120.329 224.938,128.46 226.912,136.592 228.886,144.696 230.861,152.748 232.835,160.725 234.809,168.604 
 236.783,176.368 238.757,183.998 240.731,191.481 242.705,198.802 244.679,205.951 246.654,212.916 248.628,219.691 250.602,226.268 252.576,232.641 254.55,238.807 
 256.524,244.763 258.498,250.507 260.472,256.037 262.447,261.355 264.421,266.462 266.395,271.358 268.369,276.047 270.343,280.532 272.317,284.816 274.291,288.903 
 276.265,292.798 278.24,296.505 280.214,300.031 282.188,303.379 284.162,306.556 286.136,309.568 288.11,312.42 290.084,315.118 292.058,317.667 294.033,320.075 
 296.007,322.347 297.981,324.488 299.955,326.504 301.929,328.402 303.903,330.186 305.877,331.862 307.852,333.435 309.826,334.911 311.8,336.294 313.774,337.59 
 315.748,338.803 317.722,339.937 319.696,340.997 321.67,341.987 323.645,342.911 325.619,343.773 327.593,344.577 329.567,345.325 331.541,346.022 333.515,346.671 
 335.489,347.274 337.463,347.834 339.438,348.355 341.412,348.838 343.386,349.287 345.36,349.702 347.334,350.088 349.308,350.445 351.282,350.775 353.256,351.081 
 355.231,351.364 357.205,351.626 359.179,351.867 361.153,352.091 363.127,352.297 365.101,352.487 367.075,352.662 369.049,352.824 371.024,352.973 372.998,353.11 
 374.972,353.237 376.946,353.353 378.92,353.46 380.894,353.558 382.868,353.649 384.843,353.732 386.817,353.808 388.791,353.879 390.765,353.943 392.739,354.002 
 394.713,354.056 396.687,354.106 398.661,354.152 400.636,354.193 402.61,354.232 404.584,354.267 406.558,354.299 408.532,354.328 410.506,354.355 412.48,354.379 
 414.454,354.402 416.429,354.422 418.403,354.441 420.377,354.458 422.351,354.474 424.325,354.488 426.299,354.501 428.273,354.513 430.247,354.524 432.222,354.534 
 434.196,354.543 436.17,354.551 438.144,354.559 440.118,354.565 442.092,354.572 444.066,354.577 446.04,354.582 448.015,354.587 449.989,354.591 451.963,354.595 
 453.937,354.599 455.911,354.602 457.885,354.605 459.859,354.608 461.834,354.61 463.808,354.612 465.782,354.614 467.756,354.616 469.73,354.618 471.704,354.619 
 473.678,354.62 475.652,354.622 47

In [14]:

C1, = QuadGK.quadgk(x->BSintegrand(x,S,K,y,m,σ^2),K,Inf)

printlnPs("\nCall price according to integration of risk-neutral dist: ",C1)

printlnPs("\nCompare with the result from the BS formula (from above): $(round(C,3))")


Call price according to integration of risk-neutral dist:      2.893

Compare with the result from the BS formula (from above): 2.893


# Convergence of BOPM to BS

Idea: calculate the option price according to binomial model with a CRRA calibration where $u=e^{\sigma\sqrt{h}}\text{ and }d=e^{-\sigma\sqrt{h}}$. Keeping the time to expiration $m$ fixed, try using more and more time steps ($n$), but recall that $h=m/n$

### From Chapter on Binominal Model

In [15]:
function BuildSTree(S,n,u,d)
    STree = [fill(NaN,i) for i = 1:n+1]  #container for vectors (one for each time node)
    STree[1][1] = S                       
    for i = 2:n+1                        #move forward in time
        for j = 1:i-1    
            STree[i][j] = u*STree[i-1][j]         #up move from all in STree[i-1]
        end
        STree[i][end] = d*STree[i-1][end]       #down move from STree[i-1][end] 
    end
    return STree    
end    


function EuOptionPrice(STree,K,y,n,h,u,d,p,isPut=false)     #price of American option
    Value =[fill(NaN,i) for i = 1:n+1]          #tree for derivative, to fill
    if isPut 
        Value[n+1] = max.(0,K-STree[n+1])            #put, at last time node
    else
        Value[n+1] = max.(0,STree[n+1]-K)            #call, at last time node    
    end    
    for i = n:-1:1                              #move backward in time
        for j = 1:i                             #loop over S nodes at time node i            
            fa  = exp(-y*h)*(p*Value[i+1][j] + (1-p)*Value[i+1][j+1])
            Value[i][j] = fa 
        end
    end
    return Value
end

EuOptionPrice (generic function with 2 methods)

In [16]:
m = 0.5                 #time to expiration (in years)                
y = 0.05                #interest rate (annualized) 

σ = 0.2                 #annualized std of underlying asset
S = 42.0
K = 42.0

nMax = 200

CeM = fill(NaN,nMax)
for n = 1:nMax
    h = m/n                 #time step size (in years)
    u = exp(σ*sqrt(h))
    d = exp(-σ*sqrt(h))
    p = (exp(y*h) - d)/(u-d)
    STree = BuildSTree(S,n,u,d)
    Ce = EuOptionPrice(STree,K,y,n,h,u,d,p,false)
    CeM[n] = Ce[1][1]
end   


plot(1:nMax,CeM,color=:red,label="BOPM")
plot!(1:nMax,fill(C,nMax),color=:blue,label="BS")
title!("BOPM (different n, using h=m/n) and BS call option price")
xlabel!("number of time steps (n)")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 2.75 
 
 
 3.00 
 
 
 3.25 
 
 
 BOPM (different n, using h=m/n) and BS call option price 
 
 
 number of time steps (n) 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 42.1477,368.504 44.9254,167.488 47.7031,312.627 50.4808,200.791 53.2584,292.48 56.0361,215.037 58.8138,282.146 61.5914,222.924 64.3691,275.869 
 67.1468,227.929 69.9244,271.653 72.7021,231.385 75.4798,268.628 78.2574,233.915 81.0351,266.352 83.8128,235.847 86.5904,264.577 89.3681,237.371 92.1458,263.154 
 94.9234,238.603 97.7011,261.988 100.479,239.62 103.256,261.015 106.034,240.473 108.812,260.191 111.589,241.2 114.367,259.485 117.145,241.826 119.922,258.872 
 122.7,242.372 125.478,258.335 128.255,242.85 131.033,257.861 133.811,243.274 136.588,257.44 139.366,243.652 142.144,257.062 144.921,243.991 147.699,256.723 
 150.477,244.297 153.254,256.415 156.032,244.575 158.81,256.136 161.587,244.828 164.365,255.881 167.143,245.059 169.92,255.647 172.698,245.271 175.476,255.431 
 178.253,245.467 181.031,255.232 183.809,245.647 186.586,255.048 189.364,245.815 192.142,254.877 194.919,245.971 197.697,254.718 200.475,246.116 203.252,254.569 
 206.03,246.252 208.808,254.43 211.585,246.379 214.363,254.3 217.141,246.498 219.918,254.177 222.696,246.61 225.474,254.062 228.251,246.716 231.029,253.953 
 233.807,246.815 236.584,253.85 239.362,246.91 242.14,253.753 244.918,246.999 247.695,253.661 250.473,247.083 253.251,253.573 256.028,247.163 258.806,253.49 
 261.584,247.24 264.361,253.411 267.139,247.312 269.917,253.336 272.694,247.381 275.472,253.265 278.25,247.447 281.027,253.196 283.805,247.51 286.583,253.131 
 289.36,247.571 292.138,253.068 294.916,247.628 297.693,253.008 300.471,247.683 303.249,252.951 306.026,247.736 308.804,252.896 311.582,247.787 314.359,252.843 
 317.137,247.836 319.915,252.792 322.692,247.883 325.47,252.743 328.248,247.928 331.025,252.696 333.803,247.971 336.581,252.651 339.358,248.013 342.136,252.607 
 344.914,248.053 347.691,252.565 350.469,248.092 353.247,252.524 356.024,248.13 358.802,252.485 361.58,248.166 364.357,252.447 367.135,248.201 369.913,252.411 
 372.69,248.235 375.468,252.375 378.246,248.268 381.023,252.341 383.801,248.299 386.579,252.308 389.356,248.33 392.134,252.276 394.912,248.36 397.689,252.244 
 400.467,248.389 403.245,252.214 406.022,248.417 408.8,252.185 411.578,248.444 414.355,252.156 417.133,248.47 419.911,252.129 422.688,248.496 425.466,252.102 
 428.244,248.521 431.021,252.076 433.799,248.545 436.577,252.05 439.354,248.568 442.132,252.026 444.91,248.591 447.687,252.002 450.465,248.613 453.243,251.978 
 456.02,248.635 458.798,251.956 461.576,248.656 464.353,251.933 467.131,248.677 469.909,251.912 472.686,248.697 475.464,251.891 478.242,248.716 481.019,251.87 
 483.797,248.735 486.575,251.85 489.352,248.754 492.13,251.831 494.908,248.772 497.685,251.812 500.463,248.79 503.241,251.793 506.018,248.807 508.796,251.775 
 511.574,248.824 514.351,251.757 517.129,248.84 519.907,251.74 522.684,248.856 525.462,251.723 528.24,248.872 531.017,251.706 533.795,248.887 536.573,251.69 
 539.35,248.903 542.128,251.674 544.906,248.917 547.683,251.659 550.461,248.932 553.239,251.644 556.016,248.946 558.794,251.629 561.572,248.959 564.349,251.614 
 567.127,248.973 569.905,251.6 572.682,248.986 575.46,251.586 578.238,248.999 581.015,251.572 583.793,249.012 586.571,251.559 589.348,249.024 592.126,251.546 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,250.247 42.1477,250.247 44.9254,250.247 47.7031,250.247 50.4808,250.247 53.2584,250.247 56.0361,250.247 58.8138,250.247 61.5914,250.247 64.3691,250.247 
 67.1468,250.247 69.9244,250.247 72.7021,250.247 75.4798,250.247 78.2574,250.247 81.0351,250.247 83.8128,250.247 86.5904,2

# Hedging an Option

"Delta hedging" is based on the idea that we can approximate the change in the option price by 

$C_{t+h}-C_{t}\approx\frac{\partial C_{t}}{\partial S}\left(  S_{t+h}-S_{t}\right)$


Delta of a call option

$\Delta=\frac{\partial C}{\partial S}=e^{-\delta m}\Phi\left(  d_{1}\right)$ 

Delta of a put option is 

$\frac{\partial P}{\partial S}=e^{-\delta m}[\Phi\left(  d_{1}\right)-1]$

In [17]:
function OptionGreeks(S,K,m,y,σ,δ=0,PutIt=false)
    #Calculates Black-Scholes delta
    d1 = ( log.(S./K) + (y-δ+0.5*σ^2)*m ) / (σ*sqrt(m))
    d2 = d1 - σ*sqrt(m)
    if PutIt 
        Δ = exp(-δ*m)*(Φ(d1)-1)
    else     
        Δ = exp(-δ*m)*Φ(d1)              
    end    
    return Δ
end

OptionGreeks (generic function with 3 methods)

In [18]:
Δc = OptionGreeks(S,K,m,y,σ)
Δp = OptionGreeks(S,K,m,y,σ,0,true)
printlnPs("Δ of call and put and the difference: ",[Δc Δp (Δc-Δp)])

Δ of call and put and the difference:      0.598    -0.402     1.000


In [19]:
SM = 30:60
Δc_S = OptionGreeks(SM,K,m,y,σ)          #at different spot prices

plot(SM,Δc_S,color=:red,legend=false)
title!("Black-Scholes call option delta (different S)")
xlabel!("asset price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 35 
 
 
 40 
 
 
 45 
 
 
 50 
 
 
 55 
 
 
 60 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 Black-Scholes call option delta (different S) 
 
 
 asset price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 57.7953,364.014 76.2205,357.319 94.6457,347.935 113.071,335.503 131.496,319.869 149.921,301.123 168.346,279.613 186.772,255.915 205.197,230.767 
 223.622,204.992 242.047,179.411 260.472,154.769 278.898,131.682 297.323,110.602 315.748,91.813 334.173,75.4356 352.598,61.4558 371.024,49.7529 389.449,40.133 
 407.874,32.3589 426.299,26.1758 444.724,21.3309 463.15,17.5873 481.575,14.7324 500,12.5817 518.425,10.9802 536.85,9.80041 555.276,8.94013 573.701,8.31878 
 592.126,7.87402 
 "/>